In [ ]:
!pip install opendatasets

In [ ]:
pip install pandas

In [ ]:
#Install dataset

import opendatasets as od

dataset_url = 'https://www.kaggle.com/datasets/andrewmvd/malignant-lymphoma-classification'
od.download(dataset_url)

Skipping, found downloaded files in "./malignant-lymphoma-classification" (use force=True to force download)


In [ ]:
#Install PyTorch

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
#Imports

import os
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split

#Path for dataset

dataset_dir = './malignant-lymphoma-classification'

#Data Augmentation to fix the overfitting
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = ImageFolder(root=dataset_dir, transform=transform)
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
#Load Datasets

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch.nn as nn
import torchvision.models as models

num_classes = 3

model = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

In [ ]:
#Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 10
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
since = time.time()

In [ ]:


for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_loader
        else:
            model.eval()
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    print()

time_elapsed = time.time() - since
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
print(f'Best val Acc: {best_acc:.4f}')

model.load_state_dict(best_model_wts)

Epoch 1/10
----------
train Loss: 1.2528 Acc: 0.5351
val Loss: 4.7730 Acc: 0.4533

Epoch 2/10
----------
train Loss: 0.7559 Acc: 0.6756
val Loss: 1.6534 Acc: 0.5333

Epoch 3/10
----------
train Loss: 0.6796 Acc: 0.6990
val Loss: 1.5045 Acc: 0.6000

Epoch 4/10
----------
train Loss: 0.6040 Acc: 0.7157
val Loss: 1.6593 Acc: 0.4133

Epoch 5/10
----------
train Loss: 0.6437 Acc: 0.7358
val Loss: 1.5267 Acc: 0.3600

Epoch 6/10
----------
train Loss: 0.5494 Acc: 0.7826
val Loss: 0.7848 Acc: 0.6933

Epoch 7/10
----------
train Loss: 0.5055 Acc: 0.7893
val Loss: 1.2271 Acc: 0.7200

Epoch 8/10
----------
train Loss: 0.4778 Acc: 0.8094
val Loss: 0.6848 Acc: 0.7867

Epoch 9/10
----------
train Loss: 0.3225 Acc: 0.8863
val Loss: 1.2418 Acc: 0.6267

Epoch 10/10
----------
train Loss: 0.4702 Acc: 0.8227
val Loss: 0.6684 Acc: 0.6400

Training complete in 2m 56s
Best val Acc: 0.7867


<All keys matched successfully>